In [1]:
from bluesky import RunEngine

RE = RunEngine({})

from bluesky.callbacks.best_effort import BestEffortCallback
bec = BestEffortCallback()

# Send all metadata/data captured to the BestEffortCallback.
RE.subscribe(bec)

#Get the databroker https://gist.github.com/untzag/53281819709b1058c7708a250cbd3676
from databroker.v2 import temp
db = temp()

# Insert all metadata/data captured into db.
RE.subscribe(db.v1.insert)

1

In [3]:
from ophyd import Device, EpicsSignal, EpicsSignalRO
from ophyd import Component as Cpt
class MyDetector(Device):
    
    count =  Cpt(EpicsSignal,'sensor4:getCount') 
    
my_det = MyDetector('EMILEL:TestIOC00:', name='my_det')

In [7]:
from ophyd.status import SubscriptionStatus, wait

class MyMotor(Device):
    
    start_pos = Cpt(EpicsSignal,'axis4:setStartPos') 
    end_pos = Cpt(EpicsSignal,'axis4:setEndPos')
    velocity = Cpt(EpicsSignal,'axis4:setVel')
    pos = Cpt(EpicsSignalRO, 'axis4:getPos', kind='hinted')
    start_cmd = Cpt(EpicsSignal, 'axis4:start.PROC')
    init_cmd = Cpt(EpicsSignal, 'axis4:init.PROC')
    done = Cpt(EpicsSignal, 'axis4:done')
    ready = Cpt(EpicsSignal, 'axis4:ready')
    
    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self._acquiring = False
        self._paused = False
    
    def kickoff(self):
        
        
        def check_value(*,old_value, value, **kwargs):
            #Return True when the acquisition is complete, False otherwise.                           
            if not self._acquiring:  #But only report done if acquisition was already started 
                return False

            return (value != 0)
        
        self.init_cmd.put(1)
        self._acquiring = True
        
        wait(SubscriptionStatus(self.ready, check_value,settle_time=0.2))
        
        self._start_time = ttime.time()
        
        self._paused = False
        self.start_cmd.put(1)
              
        
        
        # Add a subscription status to the done signal
        st = SubscriptionStatus(self.done, check_value)
        
        return st
    

    def complete(self):
        '''Acquisition completed'''
        if not self._acquiring:
            raise RuntimeError('Not acquiring')

        self._acquiring = False
        self._paused = False
       
        # Data is ready immediately because we wait for kickoff to return done
        st = DeviceStatus(self)
        st.set_finished()
        return st

    def collect(self):
        yield {'data':{}, 'timestamps':{}, 'time':time.time()}
    
    def describe_collect(self):
        return {self.name: {}}


flyer = MyMotor('EMILEL:TestIOC00:', name="flyer")


In [8]:
# now we will try and make somthing based on count

In [12]:
import sys
import inspect
from itertools import chain, zip_longest
from functools import partial
import collections
from collections import defaultdict
import time

import numpy as np
try:
    # cytools is a drop-in replacement for toolz, implemented in Cython
    from cytools import partition
except ImportError:
    from toolz import partition

from bluesky.plans import plan_patterns

from bluesky.plans import utils
from bluesky.utils import Msg

from bluesky import preprocessors as bpp
from bluesky import plan_stubs as bps


def count(detectors, num=1, delay=None, *, per_shot=None, md=None):
    """
    Take one or more readings from detectors.

    Parameters
    ----------
    detectors : list
        list of 'readable' objects
    num : integer, optional
        number of readings to take; default is 1

        If None, capture data until canceled
    delay : iterable or scalar, optional
        Time delay in seconds between successive readings; default is 0.
    per_shot : callable, optional
        hook for customizing action of inner loop (messages per step)
        Expected signature ::

           def f(detectors: Iterable[OphydObj]) -> Generator[Msg]:
               ...

    md : dict, optional
        metadata

    Notes
    -----
    If ``delay`` is an iterable, it must have at least ``num - 1`` entries or
    the plan will raise a ``ValueError`` during iteration.
    """
    if num is None:
        num_intervals = None
    else:
        num_intervals = num - 1
    _md = {'detectors': [det.name for det in detectors],
           'num_points': num,
           'num_intervals': num_intervals,
           'plan_args': {'detectors': list(map(repr, detectors)), 'num': num},
           'plan_name': 'count',
           'hints': {}
           }
    _md.update(md or {})
    _md['hints'].setdefault('dimensions', [(('time',), 'primary')])

    if per_shot is None:
        per_shot = bps.one_shot

    @bpp.stage_decorator(detectors)
    @bpp.run_decorator(md=_md)
    def inner_count():
        return (yield from bps.repeat(partial(per_shot, detectors),
                                      num=num, delay=delay))

    return (yield from inner_count())

In [ ]:
def fly_count(detectors, flyers, *args, delay=None, per_shot=None, md=None):
    """
    Take one or more readings from detectors while flying.

    Parameters
    ----------
    detectors : list
        list of 'readable' objects
        
    flyer : 
        objects that support the flyer interface
    num : integer, optional
        number of readings to take; default is 1

        If None, capture data until canceled
    delay : iterable or scalar, optional
        Time delay in seconds between successive readings; default is 0.
    per_shot : callable, optional
        hook for customizing action of inner loop (messages per step)
        Expected signature ::

           def f(detectors: Iterable[OphydObj]) -> Generator[Msg]:
               ...

    md : dict, optional
        metadata

    Notes
    -----
    If ``delay`` is an iterable, it must have at least ``num - 1`` entries or
    the plan will raise a ``ValueError`` during iteration.
    """
    if num is None:
        num_intervals = None
    else:
        num_intervals = num - 1
    _md = {'detectors': [det.name for det in detectors],
           'num_points': num,
           'num_intervals': num_intervals,
           'plan_args': {'detectors': list(map(repr, detectors)), 'num': num},
           'plan_name': 'count',
           'hints': {}
           }
    _md.update(md or {})
    _md['hints'].setdefault('dimensions', [(('time',), 'primary')])

    if per_shot is None:
        per_shot = bps.one_shot

    @bpp.stage_decorator(detectors)
    @bpp.run_decorator(md=_md)
    def inner_count():
        return (yield from bps.repeat(partial(per_shot, detectors),
                                      num=num, delay=delay))

    return (yield from inner_count())

In [31]:
from bluesky.plans import fly

from bluesky.preprocessors import fly_during_wrapper

# if this next step succeeds, it's proof that we did this right!
RE(fly_during_wrapper(count([my_det]), [flyer]))



Transient Scan ID: 4     Time: 2021-07-10 08:46:55
Persistent Unique Scan ID: '280760ad-94f9-491d-805d-c52ef50f9aff'


Run aborted
Traceback (most recent call last):
  File "/opt/tljh/user/lib/python3.7/site-packages/bluesky/run_engine.py", line 1365, in _run
    msg = self._plan_stack[-1].send(resp)
  File "/opt/tljh/user/lib/python3.7/site-packages/bluesky/preprocessors.py", line 861, in fly_during_wrapper
    return (yield from plan2)
  File "/opt/tljh/user/lib/python3.7/site-packages/bluesky/preprocessors.py", line 170, in plan_mutator
    raise ex
  File "/opt/tljh/user/lib/python3.7/site-packages/bluesky/preprocessors.py", line 123, in plan_mutator
    msg = plan_stack[-1].send(ret)
  File "/opt/tljh/user/lib/python3.7/site-packages/bluesky/preprocessors.py", line 170, in plan_mutator
    raise ex
  File "/opt/tljh/user/lib/python3.7/site-packages/bluesky/preprocessors.py", line 123, in plan_mutator
    msg = plan_stack[-1].send(ret)
  File "/opt/tljh/user/lib/python3.7/site-packages/bluesky/plans.py", line 77, in count
    return (yield from inner_count())
  File "/opt/tljh/user/lib/python3.7/si

TimeoutError: axis4:init.PROC could not connect within 1.0-second timeout.